# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd

df = pd.read_csv('Sentiment140.csv')
df

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [2]:
df_sampled = df.sample(n= 20000, random_state = 1)
df_sampled

,target,ids,date,flag,user,text
514293,0,2190584004,Tue Jun 16 03:08:48 PDT 2009,NO_QUERY,Vicki_Gee,i miss nikki nu nu already shes always there ...
142282,0,1881451988,Fri May 22 04:42:15 PDT 2009,NO_QUERY,PatCashin,So I had a dream last night. I remember a sig...
403727,0,2058252964,Sat Jun 06 14:34:17 PDT 2009,NO_QUERY,deelectable,@girlyghost ohh poor sickly you (((hugs)) ho...
649503,0,2237307600,Fri Jun 19 05:34:22 PDT 2009,NO_QUERY,justinekepa,it is raining again
610789,0,2224301193,Thu Jun 18 09:20:06 PDT 2009,NO_QUERY,cmatt007,@MissKeriBaby wish I was in LA right now
...,...,...,...,...,...,...
1200592,4,1985525470,Sun May 31 17:16:16 PDT 2009,NO_QUERY,cydconverse,I get the worst writer's cramp. These thank yo...
333050,0,2013234007,Tue Jun 02 22:12:13 PDT 2009,NO_QUERY,k8janfaza,@ladyinreddress the sun is all gone now...
893698,4,1691979761,Sun May 03 19:01:16 PDT 2009,NO_QUERY,fadyfresh,"FINALLY a break till Wed, no work OR school, b..."
1145456,4,1977778758,Sat May 30 21:41:26 PDT 2009,NO_QUERY,xCoreBorex,I want a Blackberry...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [3]:
#First we have to get the functions

from challenge_1 import clean_up
from challenge_1 import tokenize
from challenge_1 import stem_and_lemmatize
from challenge_1 import remove_stopwords

In [4]:
# Then we first what to try them on the sampled data 

df_sampled['text_processed'] = df_sampled['text'].apply(clean_up)
df_sampled['text_processed'] = df_sampled['text_processed'].apply(tokenize)
df_sampled['text_processed'] = df_sampled['text_processed'].apply(stem_and_lemmatize)
df_sampled['text_processed'] = df_sampled['text_processed'].apply(remove_stopwords)
df_sampled

,target,ids,date,flag,user,text,text_processed
514293,0,2190584004,Tue Jun 16 03:08:48 PDT 2009,NO_QUERY,Vicki_Gee,i miss nikki nu nu already shes always there ...,"[miss, nikki, nu, nu, alreadi, alway, need, th..."
142282,0,1881451988,Fri May 22 04:42:15 PDT 2009,NO_QUERY,PatCashin,So I had a dream last night. I remember a sig...,"[dream, last, night, ., rememb, sign, clearli,..."
403727,0,2058252964,Sat Jun 06 14:34:17 PDT 2009,NO_QUERY,deelectable,@girlyghost ohh poor sickly you (((hugs)) ho...,"[girlyghost, ohh, poor, sickli, hug, hope, fee..."
649503,0,2237307600,Fri Jun 19 05:34:22 PDT 2009,NO_QUERY,justinekepa,it is raining again,[rain]
610789,0,2224301193,Thu Jun 18 09:20:06 PDT 2009,NO_QUERY,cmatt007,@MissKeriBaby wish I was in LA right now,"[misskeribabi, wish, wa, la, right]"
...,...,...,...,...,...,...,...
1200592,4,1985525470,Sun May 31 17:16:16 PDT 2009,NO_QUERY,cydconverse,I get the worst writer's cramp. These thank yo...,"[get, worst, writer, cramp, ., thank, note, ki..."
333050,0,2013234007,Tue Jun 02 22:12:13 PDT 2009,NO_QUERY,k8janfaza,@ladyinreddress the sun is all gone now...,"[ladyinreddress, sun, gone]"
893698,4,1691979761,Sun May 03 19:01:16 PDT 2009,NO_QUERY,fadyfresh,"FINALLY a break till Wed, no work OR school, b...","[final, break, till, wed, work, school, hectic..."
1145456,4,1977778758,Sat May 30 21:41:26 PDT 2009,NO_QUERY,xCoreBorex,I want a Blackberry...,[want]


In [5]:
# Now we want to do the same for all the data

df['text_processed'] = df['text'].apply(clean_up)
df['text_processed'] = df['text_processed'].apply(tokenize)
df['text_processed'] = df['text_processed'].apply(stem_and_lemmatize)
df['text_processed'] = df['text_processed'].apply(remove_stopwords)
df

,target,ids,date,flag,user,text,text_processed
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","[switchfoot, awww, bummer, ., shoulda, got, da..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,"[upset, updat, hi, facebook, text, might, cri,..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,"[kenichan, dive, mani, time, ball, ., manag, s..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"[whole, bodi, feel, itchi, like, fire]"
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","[nationwideclass, behav, ., mad, ., whi, becau..."
...,...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,"[woke, ., school, best, feel, ever]"
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,"[veri, cool, hear, old, walt, interview, bmta]"
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,"[readi, mojo, makeov, ask, detail]"
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,"[happi, th, birthday, boo, alll, time, tupac, ..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [6]:
import nltk
from nltk.probability import FreqDist

# Step 1: Combine all words into a single list
all_words_sampled = [word for sublist in df_sampled['text_processed'] for word in sublist]

# Step 2: Calculate the frequency distribution
freq_dist_sampled = FreqDist(all_words_sampled)

# Step 3: Select the top 5,000 words by frequency
top_5000_words_sampled = [word for word, frequency in freq_dist_sampled.most_common(5000)]

# Print the top 10 words as a sample
print("Top 10 words:", top_5000_words_sampled[:10])

Top 10 words: ['.', 'go', 'get', 'wa', 'day', 'thi', 'good', 'like', 'love', 'work']


In [7]:
# Step 1: Combine all words into a single list
all_words = [word for sublist in df['text_processed'] for word in sublist]

# Step 2: Calculate the frequency distribution
freq_dist = FreqDist(all_words)

# Step 3: Select the top 5,000 words by frequency
top_5000_words = [word for word, frequency in freq_dist.most_common(5000)]

# Print the top 10 words as a sample
print("Top 10 words:", top_5000_words[:10])

Top 10 words: ['.', 'go', 'get', 'day', 'wa', 'thi', 'good', 'work', 'like', 'love']


### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [8]:
def find_features(document):
    words = set(document)
    features = {}
    for w in top_5000_words_sampled:
        features[w] = (w in words)

    return features

# Apply the find_features function to each document in the text_processed column
features = df_sampled['text_processed'].apply(find_features).tolist()

# Prepare the features and labels for the classifier
labels= (df_sampled['target']==4).tolist()
feature_set = [(feature, label) for feature, label in zip(features, labels)]

# Example output
print("Feature Set Example (first item):", feature_set[:1])



Feature Set Example (first item): [({'.': False, 'go': False, 'get': False, 'wa': False, 'day': False, 'thi': False, 'good': False, 'like': False, 'love': False, 'work': False, 'got': False, 'u': True, 'quot': False, 'miss': True, 'time': False, 'want': False, 'back': False, 'today': False, 'thank': True, 'lol': False, 'one': False, 'realli': False, 'know': False, 'think': False, 'amp': False, 'im': False, 'see': False, 'feel': False, 'watch': False, 'need': True, 'still': False, 'make': False, 'hope': False, 'night': False, 'well': False, 'oh': False, 'new': False, 'look': False, 'na': False, 'home': False, 'ha': False, 'come': False, 'much': False, 'last': False, 'twitter': False, 'wish': False, 'great': False, 'morn': False, 'wait': False, 'would': False, 'sad': False, 'tomorrow': False, 'right': False, 'sleep': False, 'whi': False, 'haha': False, 'follow': False, 'thing': False, 'onli': False, 'fun': False, 'happi': False, 'friend': False, 'tri': False, 'veri': False, 'nice': False

In [10]:
# I could not make it work for the whole dataframe. It took forever so I stopp trying for now. 
# I did manage to make the code work for the sampled data.

def find_features(document):
    words = set(document)
    features = {}
    for w in top_5000_words:
        features[w] = (w in words)

    return features

# Apply the find_features function to each document in the text_processed column
features = df['text_processed'].apply(find_features).tolist()

# Prepare the features and labels for the classifier
labels = (df['target']==4).tolist()
feature_set = [(feature, label) for feature, label in zip(features, labels)]

# Example output
print("Feature Set Example (first item):", feature_set[:1])

MemoryError: 

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [11]:
from sklearn.model_selection import train_test_split
training_set, testing_set = train_test_split(feature_set, test_size=0.2, random_state=42)

In [12]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
classifier.show_most_informative_features()


Most Informative Features
                  cancel = True            False : True   =     21.9 : 1.0
                 depress = True            False : True   =     18.5 : 1.0
                     sad = True            False : True   =     17.8 : 1.0
                   broke = True            False : True   =     15.2 : 1.0
                    ouch = True            False : True   =     13.1 : 1.0
                    lone = True            False : True   =     12.5 : 1.0
                 dentist = True            False : True   =     11.1 : 1.0
                 appreci = True             True : False  =     10.9 : 1.0
                  welcom = True             True : False  =     10.0 : 1.0
                    neck = True            False : True   =      9.8 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [13]:
nltk.classify.accuracy(classifier, testing_set)*100

71.35000000000001

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here